In [1]:
import os
import glob
import geemap
import ee, datetime
from IPython.display import Image
import pandas as pd
import geopandas as gpd
from pylab import *
import seaborn as sns
import folium
import matplotlib.pyplot as plt
%matplotlib inline
import altair as alt

In [2]:
# Setting up Google Earth Engine (GEE)
# Getting tokens and activate EE
ee.Authenticate()

# Start up this session
ee.Initialize()

Enter verification code: 4/1AWtgzh779L-enRrLkGSJsQ0tLtAVnkxpN6beVxFx52ND8SS2SziPaP6lFJk

Successfully saved authorization token.


## Masking clouds

### Sentinel 2

In [3]:
# Using QA band to filter clouds
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0)
  mask2 = qa.bitwiseAnd(cirrusBitMask).eq(0)
  mask3 = qa.bitwiseAnd(cloudBitMask).eq(0).bitwiseAnd(cirrusBitMask).eq(0)

  return image.updateMask(mask).updateMask(mask2)

### Landsat 8

In [4]:
def maskL8clouds(image):
  qa = image.select('pixel_qa')

  # Bits 3 and 5 are clouds and cloud shadows.
  cloudBitMask = 1 << 3
  cloudShadowBitMask = 1 << 5

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0)
  mask2 = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
  mask3 = qa.bitwiseAnd(cloudBitMask).eq(0).bitwiseAnd(cloudShadowBitMask).eq(0)

  return image.updateMask(mask).updateMask(mask2)

### Landsat 7

In [5]:
def maskL7clouds(image):
  qa = image.select('pixel_qa')

  # Bits 4 and 6 are clouds and cloud shadows.
  cloudBitMask = 1 << 4
  cloudShadowBitMask = 1 << 6

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0)
  mask2 = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
  mask3 = qa.bitwiseAnd(cloudBitMask).eq(0).bitwiseAnd(cloudShadowBitMask).eq(0)

  return image.updateMask(mask).updateMask(mask2)

# NDVI Bands Setup

### Sentinel 2

In [6]:
# NDVI for Sentine 2 bands
def senNDVI(image):
    return image.normalizedDifference(['B8', 'B4'])

### Landsat 8

In [7]:
# NDVI for Landsat 8 bands
def l8NDVI(image):
    return image.normalizedDifference(['B5', 'B4'])

### Landsat 7

In [8]:
# NDVI for Landsat 7 bands
def l7NDVI(image):
    return image.normalizedDifference(['B4', 'B3'])

# Set study area

In [9]:
# Rim Fire, CA
coords = [-119.634, 37.824]

# Create earth engine point
point = {'type':'Point', 'coordinates':coords}

# Time Period: Before Fire and its NDVI Analysis

In [10]:
# Set start and end date
startTime = datetime.datetime(2011, 1, 1)
endTime = datetime.datetime(2012, 12, 31)

In [12]:
# Getting L7 image collection
collection = ee.ImageCollection("LANDSAT/LE07/C01/T2").filterDate(startTime, endTime).filterBounds(point)

# Time Period: During Fire and its NDVI Analysis¶

In [ ]:
# Set start and end date
startTime = datetime.datetime(2013, 8, 1)
endTime = datetime.datetime(2012, 10, 31)

# Time Period: After Fire and its NDVI Analysis¶

In [ ]:
# Set start and end date
startTime = datetime.datetime(2020, 1, 1)
endTime = datetime.datetime(2020, 12, 31)